# Assignment 3: Question 3
#### By Hunter Gallant

---
## Pre-processing

In [3]:
f = open("A25_HW3_stardata.dat")
raw_data = f.readlines()
f.close()

In [4]:
# Processing the raw input data
# Table columns: | Type | M_V | BC | L (L_sol) | B-V |
stardata = []
for line in raw_data[1:]:
    raw_row = line.split(" ")
    row = []
    for cell in raw_row:
        if len(cell) != 0:
            row.append(cell)
        
    clean_row = []
    for i in range(len(row)):
        if i == 0: val = row[i]
        elif 0 < i < 4: val = float(row[i])
        else: val = float(row[i][:-1])
        clean_row.append(val)
    stardata.append(clean_row)
#print(stardata)

---
## Part A

$$1M_{\odot}=C\int_{0.2}^{50}m^{-1.35}dm$$

In [5]:
def makeBins(delta_m, lo, hi):
    bins = [lo]
    num_bins = int(hi/delta_m) - int(lo/delta_m)
    for i in range(num_bins):
        bins.append(round(bins[-1]+delta_m, 5))
    return bins

In [6]:
def partA(delta_m):
    bins = makeBins(delta_m, 0.2, 50)
    sum_a = 0
    for b in bins:
        sum_a += (b**(-1.35))*delta_m
    print("Sum: " + str(sum_a))
    C = 1/sum_a
    print("C: " + str(C))

In [7]:
partA(0.001)

Sum: 4.296275006090222
C: 0.2327597741258279


---
## Part B

$$L_{\textrm{bol}}= 2.328*10^9 \int_{0.2}^{50}L(m)m^{-2.35}dm$$

In [8]:
def partB(delta_m):
    bins = makeBins(delta_m, 0.2, 50)
    sum_b = 0
    for m in bins:
        # L(m) changes depending on the value of m
        if m < 0.5:
            sum_b += (0.27 * m**(2.6) * m**(-2.35)) * delta_m
        elif m < 2:
            sum_b += (m**(4.5) * m**(-2.35)) * delta_m
        elif m < 20:
            sum_b += (1.9 * m**(3.6) * m**(-2.35)) * delta_m
        else:
            sum_b += (4600 * m * m**(-2.35)) * delta_m
    front = 2.328 * 10**9
    total = front * sum_b
    return total

In [9]:
print(str(partB(0.001)/10**12) + "e+12")

4.6021571806261345e+12


---
## Part D

$$L_V = 2.328*10^9\int_{0.2}^{50}L(m)*10^{0.4BC}*m^{-2.35}dm$$
$$L_B = 2.328*10^9\int_{0.2}^{50}L(m)*10^{0.4|BC-(B-V)|}*m^{-2.35}dm$$

In [10]:
from scipy.interpolate import interp1d
import numpy as np

In [11]:
# Table columns: | Type | M_V | BC | L (L_sol) | B-V |
def interpolation():
    # Use BC to L relationship to interpolate for all values of L
    
    bc_values = []
    l_values = []
    bv_values = []
    
    for row in stardata:
        bc_values.append(row[2])
        l_values.append(row[3])
        bv_values.append(row[4])
    
    # SciPy interpolation --> interp1d
    # https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html
    interpolate_bc = interp1d(l_values, bc_values, fill_value="extrapolate")
    interpolate_bv = interp1d(l_values, bv_values, fill_value="extrapolate")
    
    # return the interpolation functions so this function only needs to get called once
    return interpolate_bc, interpolate_bv

In [12]:
def partD(delta_m):
    bins = makeBins(delta_m, 0.2, 50)
    # sums of V and B bands
    sum_v = 0
    sum_b = 0
    
    # only call interpolation once
    int_bc, int_bv = interpolation()
    
    for m in bins:
        # L(m) changes depending on the value of m
        if m < 0.5:
            L = 0.27 * m**(2.6)
        elif m < 2:
            L = m**(4.5)
        elif m < 20:
            L = 1.9 * m**(3.6)
        else:
            L = 4600 * m
        # BC and (B-V) interpolation
        temp_bc = int_bc([L])
        temp_bv = int_bv([L])
        #print(temp_bc, temp_bv, abs(temp_bc-temp_bv))
        sum_v += L * 10**(0.4 * temp_bc) * m**(-2.35) * delta_m
        sum_b += L * 10**(0.4 * (temp_bc-temp_bv)) * m**(-2.35) * delta_m 
    
    front = 2.328 * 10**9
    total_v = front * sum_v
    total_b = front * sum_b
    
    return total_v, total_b

In [13]:
total_v, total_b = partD(0.001)
print(total_v)
print(total_b)

[3.36246889e+11]
[4.21289719e+11]


In [14]:
import math

In [15]:
diff_b_v = 2.5 * math.log((total_b)/(total_v),10)
print(diff_b_v)

0.2448064640483531


---
## Part E

$$1=\int_{0.08}^{0.5} C_1 m^{-1.3}$$
$$1=\int_{0.5}^{120} C_2 m^{-2.3}$$

In [16]:
# getting the new coefficients for N(m) values
def nM(delta_m, lo, hi, exp):
    bins = makeBins(delta_m, lo, hi)
    sum_a = 0
    for b in bins:
        sum_a += b**(exp + 1)*delta_m
    print("Sum: " + str(sum_a))
    C = 1/sum_a
    print("C: " + str(C))

In [17]:
nM(0.001, 0.08, 0.5, -1.3)
nM(0.001, 0.5, 120, -2.3)

Sum: 0.6372542025339074
C: 1.5692324915609974
Sum: 3.3123190637179256
C: 0.3019032830966308


$$L_V = 10^{10}\bigg{(}1.569\int_{0.2}^{0.5}L(m)*10^{0.4BC}*m^{-1.3}dm + 0.302\int_{0.5}^{50}0.302L(m)*10^{0.4BC}*m^{-2.3}dm\bigg{)}$$
$$L_B = 10^{10}\bigg{(}1.569\int_{0.2}^{0.5}L(m)*10^{0.4[BC-(B-V)]}*m^{-1.3}dm + 0.302\int_{0.5}^{50}L(m)*10^{0.4[BC-(B-V)]}*m^{-2.3}dm\bigg{)}$$

In [18]:
def partE(delta_m):
    bins = makeBins(delta_m, 0.2, 50)
    # sums of V and B bands
    sum_v = 0
    sum_b = 0
    
    # only call interpolation once
    int_bc, int_bv = interpolation()
    
    for m in bins:
        # L(m) changes depending on the value of m
        if m < 0.5:
            L = 0.27 * m**(2.6)
        elif m < 2:
            L = m**(4.5)
        elif m < 20:
            L = 1.9 * m**(3.6)
        else:
            L = 4600 * m
        # BC and (B-V) interpolation
        temp_bc = int_bc([L])
        temp_bv = int_bv([L])
        
        # N(m) changes at 0.5
        if m < 0.5:
            nM = 1.569 * m**(-1.3)
        else:
            nM = 0.302 * m**(-2.3)
        
        sum_v += L * 10**(0.4 * temp_bc) * nM * delta_m
        sum_b += L * 10**(0.4 * (temp_bc-temp_bv)) * nM * delta_m 
    
    front = 10**10
    total_v = front * sum_v
    total_b = front * sum_b
    
    return total_v, total_b

In [19]:
e_total_v, e_total_b = partE(0.001)
print(e_total_v)
print(e_total_b)

[4.98215955e+11]
[6.26298014e+11]


In [20]:
e_diff_b_v = 2.5 * math.log(e_total_b/e_total_v,10)
print(e_diff_b_v)

0.24840850820350177
